# Website Demo for Infection Risk Calculator with Information Retrieved from Univeristy Systems

**This is the demo for our website application which estimates infection risk based on building information** 

Import necessary packages and files

In [1]:
from jupyter_dash import JupyterDash
import dash


import os

import sys
import dash_core_components as dcc
import dash_html_components as html
import dash
import sys
import os
sys_path = os.path.dirname(os.getcwd())
#for windows
src_path = sys_path + "\src"
#for mac
#src_path = sys_path + "/src"
sys.path.insert(1, src_path)
from calculator import *
assumptions = var
#for windows
src_path = sys_path + "\notebook"
#for mac
#src_path = sys_path + "/notebook"
sys.path.insert(1, src_path)

In [2]:
#pip install jupyter-dash # Uncomment this to Install jupyter-dash for running HTML in Python

Import data

In [2]:
rid_path = 'rm.csv'
sys_path = os.path.dirname(os.getcwd())


data_path = sys_path + "/data/"

room_df = pd.read_csv(data_path + rid_path)
rooms = room_df.copy()

rooms_id = []
for rid in room_df['Room']:
    rooms_id.append({'label': rid, 'value': rid})
    
rooms.head()

,Room,Area,Height,Windows,Capacity,C19_Occupancy,VAVmin,VAVmax
0,CENTER 101,3428,21.0,NaN,295.0,50.0,3500.0,7000.0
1,CENTER 115,2143,21.0,NaN,196.0,31.0,1290.0,4300.0
2,CENTER 119,2144,21.0,NaN,196.0,30.0,1290.0,4300.0
3,GH 242,2956,NaN,NaN,417.0,50.0,935.0,2325.0
4,LEDDN AUD,1888,NaN,NaN,216.0,30.0,4000.0,12000.0


**Build Website** 

In [5]:
vav_room = {'Select...': [0,0,0]}
index = 0
for i in rooms['Room']:
    vav_room[i] = [rooms['VAVmin'][index], rooms['VAVmax'][index], (rooms['VAVmin'][index] + rooms['VAVmax'][index])/2]
    index += 1
room_names =list(vav_room.keys())

In [10]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

server = app.server
activities = [{'label':'Lecture', 'value':'Lecture'}, {'label':'Studying', 'value':'Studying'}, {'label':'Singing', 'value':'Singing'}, {'label':'Social', 'value':'Social'}, {'label':'Exercising', 'value':'Exercising'}]
app.layout = html.Div([
    html.H6("Event Information"),
    #MAKE ROOM ID A DROP DOWN?
    html.Div(["RoomID: ",
              dcc.Dropdown(id='room-dropdown', value = list(vav_room.keys())[0],options = [{'label':name, 'value':name} for name in room_names])]),
    html.Br(),
    html.Div(["VAV levels: ",
             dcc.Dropdown(id='vav-dropdown')]),
    html.Br(),
    html.Div(["Duration of Event (min): ",
              dcc.Input(id='time-input', value = 0, type='number')]),
    html.Br(),
    html.Div(["Number of Occupants: ",
              dcc.Input(id='occupant-input', value = 0, type='number')]),
    html.Br(),
    html.Div(["Activity: ",
              dcc.Dropdown(id='activity-dropdown', value ='test', options=activities)]),
    html.Br(),
    html.Div(["Masks: ",
              dcc.RadioItems(id = 'masks-radio', value = 0,
    options=[
        {'label': 'Masks', 'value': 1},
        {'label': 'No Masks', 'value': 0},
    ], labelStyle={'display': 'inline-block'}
)  ]),
#     html.Br(),
#     html.Button('Reset', id='reset-button'),
    html.Br(),
    html.Button('Go', id = 'go-button', n_clicks = 0),
    html.Br(),
    html.Div(id = 'calc-output', children = 'Enter values to calculate risk')

])

@app.callback(
    dash.dependencies.Output('vav-dropdown', 'options'),
    [dash.dependencies.Input('room-dropdown', 'value')]
)
def update_date_dropdown(name):
    return [{'label': 'cfm min', 'value': "min"}, {'label': 'current', 'value': "current"}, {'label': 'cfm max', 'value': "max"}]

@app.callback(
    dash.dependencies.Output('calc-output', 'children'),
    [dash.dependencies.Input('go-button', 'n_clicks')],
    [dash.dependencies.Input('activity-dropdown', 'value')],
    [dash.dependencies.Input('room-dropdown', 'value')],
    [dash.dependencies.Input('vav-dropdown', 'value')],
    [dash.dependencies.Input('masks-radio', 'value')],
    [dash.dependencies.State('time-input', 'value')],
    [dash.dependencies.State('occupant-input', 'value')]
)
def update_calc(n_clicks, activity_dropdown, room_input,vav_dropdown, time_input, occupant_input, mask_tf):
    if n_clicks >= 1:
        comp_ir = ui_calc(activity_dropdown, room_input, time_input, occupant_input, mask_tf, data_path + rid_path, vav_dropdown)
        total_inf = int(occupant_input * comp_ir)
        to_return = 'The risk of an individual infected because of holding a(n) {} event for {} minutes in {} is {}%, given the most recent infection rates. With {} occupants, it is likely that {} occupant(s) will be infected.'.format(activity_dropdown, 
                                                                                                                                time_input, 
                                                                                                                                room_input, 
                                                                                                                                round((comp_ir * 100),2), 
                                                                                                                                occupant_input,
                                                                                                                                total_inf)
        return to_return
    else:
        return 'Enter Values to get risk calculation' 

In [11]:
app._terminate_server_for_port("localhost", 8050)
app.run_server()

Dash app running on http://127.0.0.1:8050/
The resulting risk of infection is 0.01818953634967757%
It is predicted that 0.0001818953634967757 x 110 = 0 susceptible occupants will be infected
The resulting risk of infection is 0.35898923339678923%
It is predicted that 0.0035898923339678923 x 110 = 0 susceptible occupants will be infected
The resulting risk of infection is 12.567943138089898%
It is predicted that 0.125679431380899 x 110 = 13 susceptible occupants will be infected


In [15]:
data_path + rid_path 

'C:\\Users\\HinanawiTS\\Documents\\GitHub\\Infection-Risk-App/data/rm.csv'

In [16]:
def get_room_data(filepath, room_id):
    CUBIC_FT_TO_METERS = 0.0283168
    room_table = pd.read_csv(filepath)
    room_dic = {}
    if len(room_table.loc[room_table['Room'] == room_id]) == 0:
        return print("User input error: Room " + room_id + " not found.")
    room_table.loc[room_table['Room'] == room_id]['Area']
    #Room Area in square ft.
    room_dic['room_area'] = room_table.loc[room_table['Room'] == room_id]['Area'].item()
    #Room Height. Average room height of 10 ft is chosen if nan
    room_hght = room_table.loc[room_table['Room'] == room_id]['Height'].item()
    if np.isnan(room_hght):
        print(room_id + ' Room height not found. Average room height of 10 ft imputed.')
        room_hght = 10
    room_dic['room_hght'] = room_hght
    
    #CFM range. If no CFM is provided min is chosen by default
    cfm_min = room_table.loc[room_table['Room'] == room_id]['VAVmin'].item()
    cfm_max = room_table.loc[room_table['Room'] == room_id]['VAVmax'].item()
    
    room_dic["cfm_min"] = float(cfm_min)
    
    room_dic["cfm_max"] = float(cfm_max)
    """if isinstance(cfm_range, float):
        ca_requirement_cfm = .53 * room_dic['room_area']
        room_dic['cfm_range'] = [ca_requirement_cfm, ca_requirement_cfm]
        print(room_id + ' CFM rate not found. California minimum ventilation requirement imputed')
    else:
        room_dic['cfm_range'] = list(map(int, cfm_range.split(',')))"""
    
    #Windows
    room_dic['windows'] = room_table.loc[room_table['Room'] == room_id]['Windows'].item()
    
    #V is volume of room
    room_dic['room_volume'] = room_dic['room_area'] * room_hght
    #Unit Conversion
    room_dic['room_volume_m'] = room_dic['room_area'] * room_hght * CUBIC_FT_TO_METERS
    
    return room_dic

In [19]:
get_room_data(data_path + rid_path, "CENTER 101")

{'room_area': 3428,
 'room_hght': 21.0,
 'cfm_min': 3500.0,
 'cfm_max': 7000.0,
 'windows': nan,
 'room_volume': 71988.0,
 'room_volume_m': 2038.4697984}